## Import statements for Dataframe manipulation

In [1]:
import pandas as pd
import numpy as np
import math
import re
import itertools

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RCV Voting Method Research/Voting Ballots')

Mounted at /content/drive


# Check for spoiler effect

In [7]:
output = pd.DataFrame(columns = ["Election", "% of writeins"])

In [10]:
#List of filenames

filenames = [f for f in os.listdir('.') if os.path.isfile(f)]

for filename in filenames:

  #print(filename)

  # Read from file
  data = pd.read_csv(filename)

  # Remove NA
  data = data.dropna()

  # Remove irrelevant attributes
  irrelevantAttributes = []

  attributes = data.columns

  # identify irrelevant attributes
  pattern = re.compile("^[rR][aA][nN][kK]")

  length = len(attributes)

  for i in range(1, length + 1):
      if re.search(pattern, attributes[length - i]) == None:
        irrelevantAttributes.append(attributes[length - i])

  # drop irrelevant attributes
  data = data.drop(columns = irrelevantAttributes)
  attributes = data.columns

  # Find candidates
  candNum = data.shape[1]
  data = data.replace('^[Ww][Rr][Ii][Tt][Ee].*[Ii][Nn].*$', 'writein', regex=True)
  data = data.replace('^[Oo][Vv][Ee][Rr][Vv][Oo][Tt][Ee]', 'overvote', regex=True)
  data = data.replace('^[Oo][Vv][Ee][Rr][Vv][Oo][Tt][Ee]', 'overvote', regex=True)
  data = data.replace(['skipped', 'Undeclared'], '-')

  # Convert to numpy
  data = data.to_numpy()

  # Ignore after overvote
  for i in range(data.shape[0]):
    overvoted = False
    for j in range(data.shape[1]):
      if data[i][j] == 'overvote':
        overvoted = True

      if overvoted:
        data[i][j] = '-'

  # Ignore duplicated candidates
  for i in range(data.shape[0]):
    ballot = ["candidate entry"] * candNum
    for j in range(data.shape[1]):
      for b in range(len(ballot)):
        if data[i][j] != '-':
          if ballot[b] == "candidate entry":
            ballot[b] = data[i][j]
            break
          elif ballot[b] == data[i][j]:
            data[i][j] = '-'
            break

  # Shift candidates to group on the left
  for i in range(np.size(data, 0)):
    for j in range(np.size(data, 1)-1):
      if data[i][j] == '-':
        k = j + 1
        while k < np.size(data, 1) and data[i][k] == '-':
          k += 1
        if k < np.size(data, 1):
          temp = data[i][j]
          data[i][j] = data[i][k]
          data[i][k] = temp

  data = pd.DataFrame(data, columns = attributes)

  sum = 0
  writein = 0
  for candidate in data.rank1.unique():
    count = (data['rank1'] == candidate).sum()
    #print(candidate + ": " + str(count))
    if candidate != '-':
      sum = sum + count
    if candidate == 'writein':
      writein = count

  #print("\n" + "Total: " + str(sum))
  if sum == 0:
    ratio = 1
  else:
    ratio = float(writein) / sum
  #print("Ratio: " + str(ratio) + "\n")

  row = [filename, ratio]

  output.loc[len(output)] = row


<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)
<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)
<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)
<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)
<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)
<ipython-input-10-a3a2de7afdc5>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename

AttributeError: 'DataFrame' object has no attribute 'rank1'

In [11]:
output.to_csv("../Test.csv")